In [1]:
import sys
import mne
import pandas as pd
import os

In [2]:
sys.path.append("../")
from code.preprocessing_module import Participant

ModuleNotFoundError: No module named 'code.preprocessing_module'; 'code' is not a package